In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime → "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Thu May  7 07:27:05 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.82       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [2]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('To enable a high-RAM runtime, select the Runtime → "Change runtime type"')
  print('menu, and then select High-RAM in the Runtime shape dropdown. Then, ')
  print('re-execute this cell.')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.4 gigabytes of available RAM

You are using a high-RAM runtime!


In [3]:
from google.colab import drive
drive.mount("/content/gdrive", force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
# # !pip install tensorflow-gpu==2.0.0-alpha0
# %tensorflow_version 2.x
# !pip uninstall -y tensorflow
# !pip install tensorflow-gpu==2.0.0-alpha0

In [0]:
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn.model_selection import train_test_split
import unicodedata
import re
import numpy as np
import os
import io
import time
import numpy as np
import tensorflow as tf
import re
import time

In [0]:
path = '/content/gdrive/My Drive/Colab Notebooks/cmpe258/final-project/Version-v4/basic-seq2seq/'

In [0]:
import os
os.chdir(path)

In [7]:
!pwd

/content/gdrive/My Drive/Colab Notebooks/cmpe258/final-project/Version-v4/basic-seq2seq


In [0]:
conversations = open('data/movie_conversations.txt', encoding = 'utf-8', errors = 'ignore').read().split('\n')
lines = open('data/movie_lines.txt', encoding = 'utf-8', errors = 'ignore').read().split('\n')

In [9]:
conversations[0:5]

["u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L194', 'L195', 'L196', 'L197']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L198', 'L199']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L200', 'L201', 'L202', 'L203']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L204', 'L205', 'L206']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L207', 'L208']"]

In [0]:
id2line = {}
for line in lines:
    _line = line.split(' +++$+++ ')
    if len(_line) == 5:
        id2line[_line[0]] = _line[4]

In [0]:
conversations_ids = []
for conversation in conversations[:-1]:
    _conversation = conversation.split(' +++$+++ ')[-1][1:-1].replace("'", "").replace(" ", "")
    conversations_ids.append(_conversation.split(','))

In [0]:
questions = []
answers   = []
for conversation in conversations_ids:
    for i in range(len(conversation) - 1):
        questions.append(id2line[conversation[i]])
        answers.append(id2line[conversation[i+1]])

In [0]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r"i'm", "i am", text)
    text = re.sub(r"he's", "he is", text)
    text = re.sub(r"she's", "she is", text)
    text = re.sub(r"that's", "that is", text)
    text = re.sub(r"what's", "what is", text)
    text = re.sub(r"where's", "where is", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"won't", "will not", text)
    text = re.sub(r"can't", "cannot", text)
    text = re.sub(r"[-()\"#/@;:<>{}+=~|.?,]", "", text)
    return text

In [14]:
clean_questions = []
for question in questions:
    clean_questions.append(clean_text(question))
print(clean_questions[0])
print(len(clean_questions))

can we make this quick  roxanne korrine and andrew barrett are having an incredibly horrendous public break up on the quad  again
221616


In [15]:
clean_answers = []
for answer in answers:
    clean_answers.append(clean_text(answer))
print(clean_answers[0])
print(len(clean_answers))

well i thought we would start with pronunciation if that is okay with you
221616


In [0]:
# Converts the unicode file to ascii
def unicode_to_ascii(s):
  return ''.join(c for c in unicodedata.normalize('NFD', s)
      if unicodedata.category(c) != 'Mn')

def preprocess_sentence(w):
  w = unicode_to_ascii(w.lower().strip())
  # creating a space between a word and the punctuation following it
  # eg: "he is a boy." => "he is a boy ."
  # Reference:- https://stackoverflow.com/questions/3645931/python-padding-punctuation-with-white-spaces-keeping-punctuation
  w = re.sub(r"([?.!,¿])", r" \1 ", w)
  w = re.sub(r'[" "]+', " ", w)

  # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")
  w = re.sub(r"[^a-zA-Z?.!,¿]+", " ", w)

  w = w.strip()

  # adding a start and an end token to the sentence
  # so that the model know when to start and stop predicting.
  w = '<start> ' + w + ' <end>'
  return w

In [0]:
def create_dataset(num_examples):
  sentence_pairs = []
  for index in range(num_examples):
    sentence_pair = [(preprocess_sentence(clean_questions[index])),preprocess_sentence(clean_answers[index])]
    sentence_pairs.append(sentence_pair)
  print(len(sentence_pairs))
  print(type(sentence_pairs))
  print(sentence_pairs[0])
  print(type(sentence_pairs[0]))
  return zip(*sentence_pairs)

In [18]:
num_examples = 8000
inp_lang, targ_lang = create_dataset(num_examples)

8000
<class 'list'>
['<start> can we make this quick roxanne korrine and andrew barrett are having an incredibly horrendous public break up on the quad again <end>', '<start> well i thought we would start with pronunciation if that is okay with you <end>']
<class 'list'>


In [0]:
general_lang = inp_lang + targ_lang

In [20]:
len(general_lang)

16000

In [0]:
def tokenize(inp_lang, targ_lang, general_lang):
  lang_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='')
  lang_tokenizer.fit_on_texts(general_lang)
  
  input_tensor = lang_tokenizer.texts_to_sequences(inp_lang)
  input_tensor = tf.keras.preprocessing.sequence.pad_sequences(input_tensor, padding='post')

  target_tensor = lang_tokenizer.texts_to_sequences(targ_lang)
  target_tensor = tf.keras.preprocessing.sequence.pad_sequences(target_tensor, padding='post')

  return input_tensor, lang_tokenizer, target_tensor, lang_tokenizer

In [0]:
def load_dataset():
  input_tensor, inp_lang_tokenizer, target_tensor, targ_lang_tokenizer = tokenize(inp_lang, targ_lang, general_lang)
  return input_tensor, target_tensor, inp_lang_tokenizer, targ_lang_tokenizer

In [0]:
# def tokenize(lang, general_lang):
#   lang_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='')
#   lang_tokenizer.fit_on_texts(general_lang)
  
#   tensor = lang_tokenizer.texts_to_sequences(lang)
#   tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')
#   return tensor, lang_tokenizer

In [0]:
# def load_dataset():
#   input_tensor, inp_lang_tokenizer   = tokenize(inp_lang,  general_lang)
#   target_tensor, targ_lang_tokenizer = tokenize(targ_lang, general_lang)
#   return input_tensor, target_tensor, inp_lang_tokenizer, targ_lang_tokenizer

In [0]:
input_tensor, target_tensor, inp_lang, targ_lang = load_dataset()

In [0]:
max_length_targ, max_length_inp = target_tensor.shape[1], input_tensor.shape[1]

In [0]:
# input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(input_tensor, target_tensor, test_size=0.0001)
input_tensor_train = input_tensor
target_tensor_train = target_tensor

In [26]:
print(len(input_tensor_train), len(target_tensor_train))

8000 8000


In [0]:
def id2word(lang):
  for t in range(1, 10):
    print ("%d ----> %s" % (t, lang.index_word[t]))

In [29]:
id2word(inp_lang)

1 ----> <start>
2 ----> <end>
3 ----> you
4 ----> i
5 ----> the
6 ----> to
7 ----> a
8 ----> it
9 ----> is


In [28]:
id2word(targ_lang)

1 ----> <start>
2 ----> <end>
3 ----> you
4 ----> i
5 ----> the
6 ----> to
7 ----> a
8 ----> it
9 ----> is


In [0]:
def convert(lang, tensor):
  for t in tensor:
    if t!=0:
      print ("%d ----> %s" % (t, lang.index_word[t]))

In [31]:
print ("Input Language; index to word mapping")
convert(inp_lang, input_tensor_train[0])
print ()
print ("Target Language; index to word mapping")
convert(targ_lang, target_tensor_train[0])

Input Language; index to word mapping
1 ----> <start>
58 ----> can
21 ----> we
115 ----> make
27 ----> this
1529 ----> quick
6180 ----> roxanne
6181 ----> korrine
13 ----> and
6182 ----> andrew
6183 ----> barrett
14 ----> are
422 ----> having
85 ----> an
1908 ----> incredibly
6184 ----> horrendous
1394 ----> public
628 ----> break
55 ----> up
36 ----> on
5 ----> the
6185 ----> quad
225 ----> again
2 ----> <end>

Target Language; index to word mapping
1 ----> <start>
66 ----> well
4 ----> i
150 ----> thought
21 ----> we
40 ----> would
289 ----> start
38 ----> with
3855 ----> pronunciation
50 ----> if
11 ----> that
9 ----> is
111 ----> okay
38 ----> with
3 ----> you
2 ----> <end>


In [32]:
BUFFER_SIZE = len(input_tensor_train)
print("BUFFER_SIZE:", BUFFER_SIZE)
BATCH_SIZE = 8
steps_per_epoch = len(input_tensor_train)//BATCH_SIZE
embedding_dim = 256
units = 1024
vocab_inp_size = len(inp_lang.word_index) +1
vocab_tar_size = len(targ_lang.word_index)+1
dataset = tf.data.Dataset.from_tensor_slices((input_tensor_train, target_tensor_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

BUFFER_SIZE: 8000


In [33]:
example_input_batch, example_target_batch = next(iter(dataset))
# print(example_input_batch)
example_input_batch_numpy = example_input_batch.numpy()
convert(inp_lang, example_input_batch_numpy[1])
example_input_batch.shape, example_target_batch.shape

1 ----> <start>
4339 ----> solid
112 ----> little
1337 ----> jobs
299 ----> aren
10 ----> t
39 ----> they
2 ----> <end>


(TensorShape([8, 226]), TensorShape([8, 202]))

In [0]:
class Encoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
    super(Encoder, self).__init__()
    self.batch_sz = batch_sz
    self.enc_units = enc_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(self.enc_units, return_sequences=True, return_state=True, recurrent_initializer='glorot_uniform')

  def call(self, x, hidden):
    x = self.embedding(x)
    output, state = self.gru(x, initial_state = hidden)
    return output, state

  def initialize_hidden_state(self):
    return tf.zeros((self.batch_sz, self.enc_units))

In [35]:
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)
# sample input
sample_hidden = encoder.initialize_hidden_state()
sample_output, sample_hidden = encoder(example_input_batch, sample_hidden)
print ('Encoder output shape: (batch size, sequence length, units) {}'.format(sample_output.shape))
print ('Encoder Hidden state shape: (batch size, units) {}'.format(sample_hidden.shape))

Encoder output shape: (batch size, sequence length, units) (8, 226, 1024)
Encoder Hidden state shape: (batch size, units) (8, 1024)


In [0]:
class BahdanauAttention(tf.keras.layers.Layer):
  def __init__(self, units):
    super(BahdanauAttention, self).__init__()
    self.W1 = tf.keras.layers.Dense(units)
    self.W2 = tf.keras.layers.Dense(units)
    self.V = tf.keras.layers.Dense(1)

  def call(self, query, values):
    # query hidden state shape == (batch_size, hidden size)
    # query_with_time_axis shape == (batch_size, 1, hidden size)
    # values shape == (batch_size, max_len, hidden size)
    # we are doing this to broadcast addition along the time axis to calculate the score
    query_with_time_axis = tf.expand_dims(query, 1)

    # score shape == (batch_size, max_length, 1)
    # we get 1 at the last axis because we are applying score to self.V
    # the shape of the tensor before applying self.V is (batch_size, max_length, units)
    score = self.V(tf.nn.tanh(
        self.W1(query_with_time_axis) + self.W2(values)))

    # attention_weights shape == (batch_size, max_length, 1)
    attention_weights = tf.nn.softmax(score, axis=1)

    # context_vector shape after sum == (batch_size, hidden_size)
    context_vector = attention_weights * values
    context_vector = tf.reduce_sum(context_vector, axis=1)

    return context_vector, attention_weights

In [37]:
attention_layer = BahdanauAttention(10)
attention_result, attention_weights = attention_layer(sample_hidden, sample_output)

print("Attention result shape: (batch size, units) {}".format(attention_result.shape))
print("Attention weights shape: (batch_size, sequence_length, 1) {}".format(attention_weights.shape))

Attention result shape: (batch size, units) (8, 1024)
Attention weights shape: (batch_size, sequence_length, 1) (8, 226, 1)


In [0]:
class Decoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
    super(Decoder, self).__init__()
    self.batch_sz = batch_sz
    self.dec_units = dec_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(self.dec_units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')
    self.fc = tf.keras.layers.Dense(vocab_size)

    # used for attention
    self.attention = BahdanauAttention(self.dec_units)

  def call(self, x, hidden, enc_output):
    # enc_output shape == (batch_size, max_length, hidden_size)
    context_vector, attention_weights = self.attention(hidden, enc_output)

    # x shape after passing through embedding == (batch_size, 1, embedding_dim)
    x = self.embedding(x)

    # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)
    x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)

    # passing the concatenated vector to the GRU
    output, state = self.gru(x)

    # output shape == (batch_size * 1, hidden_size)
    output = tf.reshape(output, (-1, output.shape[2]))

    # output shape == (batch_size, vocab)
    x = self.fc(output)

    return x, state, attention_weights

In [39]:
decoder = Decoder(vocab_tar_size, embedding_dim, units, BATCH_SIZE)
sample_decoder_output, _, _ = decoder(tf.random.uniform((BATCH_SIZE, 1)), sample_hidden, sample_output)
print ('Decoder output shape: (batch_size, vocab size) {}'.format(sample_decoder_output.shape))

Decoder output shape: (batch_size, vocab size) (8, 8997)


In [0]:
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')

def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  loss_ = loss_object(real, pred)
  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask
  return tf.reduce_mean(loss_)

In [0]:
checkpoint_dir = 'cp-attention12/'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=encoder,
                                 decoder=decoder)

In [0]:
@tf.function
def train_step(inp, targ, enc_hidden):
  loss = 0

  with tf.GradientTape() as tape:
    enc_output, enc_hidden = encoder(inp, enc_hidden)

    dec_hidden = enc_hidden

    dec_input = tf.expand_dims([targ_lang.word_index['<start>']] * BATCH_SIZE, 1)

    # Teacher forcing - feeding the target as the next input
    for t in range(1, targ.shape[1]):
      # passing enc_output to the decoder
      predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)

      loss += loss_function(targ[:, t], predictions)

      # using teacher forcing
      dec_input = tf.expand_dims(targ[:, t], 1)

  batch_loss = (loss / int(targ.shape[1]))

  variables = encoder.trainable_variables + decoder.trainable_variables

  gradients = tape.gradient(loss, variables)

  optimizer.apply_gradients(zip(gradients, variables))

  return batch_loss

In [0]:
EPOCHS = 20
for epoch in range(EPOCHS):
  start = time.time()
  enc_hidden = encoder.initialize_hidden_state()
  total_loss = 0

  for (batch, (inp, targ)) in enumerate(dataset.take(steps_per_epoch)):
    batch_loss = train_step(inp, targ, enc_hidden)
    total_loss += batch_loss

    if batch % 100 == 0:
      print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1,batch, batch_loss.numpy()))
  # saving (checkpoint) the model every 2 epochs
  if (epoch + 1) % 2 == 0:
    print()
    checkpoint.save(file_prefix = checkpoint_prefix)

  print('Epoch {} Loss {:.4f}'.format(epoch + 1, total_loss / steps_per_epoch))
  print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

In [0]:
def evaluate(sentence):
  attention_plot = np.zeros((max_length_targ, max_length_inp))
  sentence = preprocess_sentence(sentence)
  print(type(inp_lang))
  inputs = [inp_lang.word_index[i] for i in sentence.split(' ')]
  inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs], maxlen=max_length_inp, padding='post')
  inputs = tf.convert_to_tensor(inputs)
  result = ''
  hidden = [tf.zeros((1, units))]
  enc_out, enc_hidden = encoder(inputs, hidden)

  dec_hidden = enc_hidden
  dec_input = tf.expand_dims([targ_lang.word_index['<start>']], 0)

  for t in range(max_length_targ):
    predictions, dec_hidden, attention_weights = decoder(dec_input, dec_hidden, enc_out)
    # storing the attention weights to plot later on
    attention_weights = tf.reshape(attention_weights, (-1, ))
    attention_plot[t] = attention_weights.numpy()
    predicted_id = tf.argmax(predictions[0]).numpy()
    result += targ_lang.index_word[predicted_id] + ' '
    if targ_lang.index_word[predicted_id] == '<end>':
      return result, sentence, attention_plot
    # the predicted ID is fed back into the model
    dec_input = tf.expand_dims([predicted_id], 0)
  return result, sentence, attention_plot

In [0]:
# function for plotting the attention weights
def plot_attention(attention, sentence, predicted_sentence):
  fig = plt.figure(figsize=(10,10))
  ax = fig.add_subplot(1, 1, 1)
  ax.matshow(attention, cmap='viridis')

  fontdict = {'fontsize': 14}

  ax.set_xticklabels([''] + sentence, fontdict=fontdict, rotation=90)
  ax.set_yticklabels([''] + predicted_sentence, fontdict=fontdict)

  ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
  ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

  plt.show()

In [0]:
def translate(sentence):
  result, sentence, attention_plot = evaluate(sentence)
  print('Input: %s' % (sentence))
  print('Predicted translation: {}'.format(result))
  # attention_plot = attention_plot[:len(result.split(' ')), :len(sentence.split(' '))]
  # plot_attention(attention_plot, sentence.split(' '), result.split(' '))

In [47]:
# restoring the latest checkpoint in checkpoint_dir
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

In [49]:
question = 'how old are you'
print(translate(question))

<class 'keras_preprocessing.text.Tokenizer'>
Input: <start> how old are you <end>
Predicted translation: that is not really a very proper question <end> 
None


In [50]:
question = 'i would like some coffee'
print(translate(question))

<class 'keras_preprocessing.text.Tokenizer'>
Input: <start> i would like some coffee <end>
Predicted translation: yeah <end> 
None


In [52]:
question = 'hi'
print(translate(question))

<class 'keras_preprocessing.text.Tokenizer'>
Input: <start> hi <end>
Predicted translation: cool pictures you a fan <end> 
None


In [53]:
question = 'what can I help you'
print(translate(question))

<class 'keras_preprocessing.text.Tokenizer'>
Input: <start> what can i help you <end>
Predicted translation: stay here if you are a great admirer or yours who wishes to amanda before <end> 
None


In [54]:
question = 'may I help you'
print(translate(question))

<class 'keras_preprocessing.text.Tokenizer'>
Input: <start> may i help you <end>
Predicted translation: you show no inclination to speak otherwise ! <end> 
None


In [55]:
question = 'what do you think about coffee'
print(translate(question))

<class 'keras_preprocessing.text.Tokenizer'>
Input: <start> what do you think about coffee <end>
Predicted translation: i don t know <end> 
None


In [56]:
question = 'what do you think about the car'
print(translate(question))

<class 'keras_preprocessing.text.Tokenizer'>
Input: <start> what do you think about the car <end>
Predicted translation: i don t know <end> 
None


In [57]:
question = 'do you like this short'
print(translate(question))

<class 'keras_preprocessing.text.Tokenizer'>
Input: <start> do you like this short <end>
Predicted translation: i am not supposed to but i am not supposed to but i am not supposed to but i am not supposed to but i am not supposed to but i am not supposed to but i am not supposed to but i am not supposed to but i am not supposed to but i am not supposed to but i am not supposed to but i am not supposed to but i am not supposed to but i am not supposed to but i am not supposed to but i am not supposed to but i am not supposed to but i am not supposed to but i am not supposed to but i am not supposed to but i am not supposed to but i am not supposed to but i am not supposed to but i am not supposed to but i am not supposed to but i am not supposed to but i am not supposed to but i am not supposed to but i am not supposed to but i am not supposed to but i am not supposed to but i am not supposed to but i am not supposed to but i am not supposed 
None


In [58]:
question = 'Can you sing a song'
print(translate(question))

<class 'keras_preprocessing.text.Tokenizer'>
Input: <start> can you sing a song <end>
Predicted translation: sure <end> 
None


In [60]:
question = 'I prefer a pop music'
print(translate(question))

<class 'keras_preprocessing.text.Tokenizer'>
Input: <start> i prefer a pop music <end>
Predicted translation: i am likin you guys better <end> 
None


In [61]:
question = 'Thank you'
print(translate(question))

<class 'keras_preprocessing.text.Tokenizer'>
Input: <start> thank you <end>
Predicted translation: there is one thing i would like to understand why do you want to try it again <end> 
None
